In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random

DEBUG = False 

# Projet: Modèles linéaires:  Adaline et Regression Logistique

Nous allons nous intéresser à l'implémentation d'un algorithme de descente de gradient pour trouver le meilleur paramètre d'un module Adaline ou de regression logistique.

Pout cela, on implémentera un algorithme de descente de gradient stochastique que nous avons vu au TP précédent et dont le pseudo-code peut être résumé comme suit:

```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    choisir un exemple (x,y) de Train de façon aléatoire
    calculer h = w*x
    calculer Loss(h, y)
    w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
output: w
```
où "eta" est le pas de la descente de gradient (exemple: eta=0.01).

Si on veut imprimer l'erreur tous les "m" pas de gradient:
```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    err = 0
    for i in range(m):
        choisir un exemple (x,y) de Train de façon aléatoire
        calculer h = w*x
        err += Loss(h, y)
        w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
    print(err)
output: w
```

Pour un poids $w$, on définit $h_\mathbf{w}(\mathbf{x})=w_0x_0+w_1x_1+...w_dx_d$. Pour chacun des deux modèles, et pour un exemple $(\mathbf{x},y)$, la prédiction $\hat{y}(\mathbf{w}, \mathbf{x})$ et la fonction de coût  $\mathcal{L}(\mathbf{w}, \mathbf{x})$ sont: 
- Adaline: $\hat{y}(\mathbf{w}, \mathbf{x}) = h_\mathbf{w}(x)$ et $$\mathcal{L}(\mathbf{w})=(y-\hat{y}(\mathbf{w},\mathbf{x}))^2=(y-h_\mathbf{w}(\mathbf{x}))^2,$$
- Régression logistique: $\hat{y}(w, x) = 1/(1+e^{-h_{\mathbf{w}}(\mathbf{x})})$ et $$\mathcal{L}(\mathbf{w}, x) = - y \log \hat{y}(\mathbf{w},\mathbf{x}) - (1-y)\log(1-\hat{y}(\mathbf{w},\mathbf{x})) = \log(1+e^{h_{\mathbf{w}}(\mathbf{x})})-yh_\mathbf{w}(\mathbf{x}),$$



In [ ]:
#  Coder Struture Adaline et Regression Logistique # 
class Model:
    model = ""
    
    def __init__(self, model):
        self.model = model    
    
    def prediction(self, w, x):
        """
        w : vecteur de poids
        x : vecteur d'entrée
        Retourne la prediction du modèle
        """
        if self.model == "Adaline":
            return w@x
        elif self.model == "Regression Logistique":
            return 1/(1 + np.exp(-w@x))
        else:
            raise ValueError("Modèle non reconnu")
    
    def loss(self, w, x, y):
        """
        p : prédiction
        y : valeur réelle
        Retourne la perte entre p et ye
        """
        if self.model == "Adaline":
            return (self.prediction(w,x)-y)**2
        elif self.model == "Regression Logistique":
            p = self.prediction(w,x)
            p = np.clip(p, 1e-10, 1-1e-10)
            return -y*np.log(p) - (1-y)*np.log(1-p)
        else:
            raise ValueError("Modèle non reconnu")

    def gradient(self, w, x, y):
        """
        w : vecteur de poids
        x : vecteur d'entrée
        y : valeur réelle
        Retourne le gradient de la fonction loss
        """
        if self.model == "Adaline":
            return 2*(self.prediction(w,x)-y)*x
        elif self.model == "Regression Logistique":
            p = self.prediction(w,x)
            return (p-y)*x
        else:
            raise ValueError("Modèle non reconnu")
    
    def __str__(self):
        return self.model

Nous avons vu les gradients de ces fonctions en TD.

## Partie 1: implémentation de l'algorithme et exemple du "ET logique"

<font color='red'><b>Question 1:</b> le "ET logique".</font> Créer une liste de 4 éléments où chaque élément est un couple de la forme `[x,y]`, avec `x=[1,x1,x2]` et `y = x1 and x2`. Il y a 4 éléments car `x1` et `x2` peuvent chacun prendre la valeur `0` ou `1` (chacun de ces 4 éléments est une liste dont le premier élément est les attributs de l'exemple et le deuxième élément est la classe de l'exemple).

In [ ]:
# Creation de liste ET logique #
liste = [0] * 4
for x1 in range(2):
    for x2 in range(2):
        liste[x1*1+x2*2] = [[1,x1,x2], x1 and x2] 

x_ET_logique = np.array([ei[0] for ei in liste])
y_ET_logique = np.array([ei[1] for ei in liste])

x_ET_logique, y_ET_logique


Codage d'une structure pour stocker les listes de feature et la liste de label 

In [ ]:
import random as rd

class collection:
    # Structure de données pour stocker les données
    def __init__(self, x, y):
        if (len(x) != len(y)):
            raise ValueError("Les données x et y n'ont pas la même longueur")
        self.x = x
        self.y = y
    
    def nombre_de_feature(self):
        return len(self.x[0])
    
    def nombre_de_data(self):
        return len(self.x)
    
    def __str__(self):
        return f"x : {self.x}, y : {self.y}"

<font color='red'><b>Question 2:</b></font> Coder un algorithme de descente de gradient stochastique pour les modèles Adaline et le modèle de régression logistique. 


In [ ]:
import random as rd

def SGD(model, collection, MaxEp, eta, m = 0):
    """
    Entree: 
    model : model Adaline ou Regression Logistique
    collection : collection de données
    eta : pas d'apprentissage
    
    Sortie:
    w : vecteur de poids
    """
    if m == 0: 
        m = 1
        err_printing = False
    else: 
        err_printing = True
    w = np.random.rand(collection.nombre_de_feature()) * 0.01
    for epoque in range(MaxEp):
        err = 0
        for _ in range(m):
            i = rd.randint(0, collection.nombre_de_data()-1)
            err += model.loss(w, collection.x[i], collection.y[i])
            w = w - eta(epoque)*model.gradient(w, collection.x[i], collection.y[i])
        if err_printing:
            print(f"Loss : {err}")
    return w
    
    
def SGD_with_error_printing(model, collection, MaxEp, m, eta):
    return SGD(model, collection, MaxEp, eta, m)


# Learning rate #   
def eta_001(t):
    return 0.01

def eta_01(t):
    return 0.1
#################

# Test de la fonction SGD
adaline = Model("Adaline")
regression_logistique = Model("Regression Logistique")
data = collection(x_ET_logique, y_ET_logique)
w = SGD(adaline, data, 100, eta_01, False)
w

Tourner les 2 modèles sur la collection de "ET logique".

In [ ]:
collection_et_logique = collection(x_ET_logique, y_ET_logique)

# Training Adaline on "ET logique"
w_adaline = SGD(adaline, collection_et_logique, 100, eta_001, False)
print("Final weights for Adaline on 'ET logique':", w_adaline)

# Training Logistic Regression on "ET logique"
w_logistic = SGD_with_error_printing(regression_logistique, collection_et_logique, 100, 3, eta_001)
print("Final weights for Logistic Regression on 'ET logique':", w_logistic)

Calculer le taux d'erreur de votre algorithme sur cette base (où une erreur est comptabilisé si la prédiction est plus proche de la fausse classe que de la vraie classe). 

In [ ]:
def calculate_error_rate(model, collection, w, rate_printing=DEBUG):
    """
    Entree:
    model : model Adaline ou Regression Logistique
    collection : collection de données
    w : vecteur de poids
    
    Sortie:
    errors : taux d'erreur
    """
    errors = 0
    n = collection.nombre_de_data()
    for i in range(n):
        pred = model.prediction(w, collection.x[i])
        if rate_printing:
            print(f"P: {pred}, Y: {collection.y[i]}")
        if (pred >= 0.5 and collection.y[i] == 0) or (pred < 0.5 and collection.y[i] == 1):
            errors += 1
    return errors / n

# Function to calculate error rate multiple times
def calculate_multiple_error_rates(model, train_collection, test_collection, num_iterations, eta, Maxep, error_rate_printing=False, debug=DEBUG):
    """
    Calculer le taux d'erreur plusieurs fois
    
    Objectif:
    - Entrainer le modèle plusieurs fois
    - Calculer le taux d'erreur à chaque itération
    - Verifier la stabilité et la convergence de l'algorithme
    """
    if str(model) == 'Adaline':
        print("Training Adaline...") 
    elif str(model) == 'Regression Logistique':
        print("Training Regression Logistique...")

    error_rates = np.zeros(num_iterations)
    for _ in range(num_iterations):
        w = SGD(model, train_collection, Maxep, eta)
        error_rate = calculate_error_rate(model, test_collection, w)
        error_rates[_] = (error_rate)
        if error_rate_printing:
            print(f"Error rate: {error_rate}")
    if debug:
        print(error_rates)
    return np.mean(error_rates)


# Calculate error rates for Adaline and Logistic Regression
num_iterations = 25

#####################################
# Calculate error rates for Adaline #
# calculate_multiple_error_rates(adaline, collection_et_logique, collection_et_logique, num_iterations, eta_001, 100, True)
# calculate_multiple_error_rates(adaline, collection_et_logique, collection_et_logique, num_iterations, eta_01, 100, True)

#################################################
# Calculate error rates for Logistic Regression #
# calculate_multiple_error_rates(regression_logistique, collection_et_logique, collection_et_logique, num_iterations, eta_001, 100, True)
# calculate_multiple_error_rates(regression_logistique, collection_et_logique, collection_et_logique, num_iterations, eta_01, 100, True)
# calculate_multiple_error_rates(regression_logistique, collection_et_logique, collection_et_logique, num_iterations, eta_01, 200, True)
# calculate_multiple_error_rates(regression_logistique, collection_et_logique, collection_et_logique, num_iterations, eta_01, 250, True)
calculate_multiple_error_rates(regression_logistique, collection_et_logique, collection_et_logique, num_iterations, eta_01, 300, True)


## Partie 2: premiers tests avec une base de donnée réelle

<font color='red'><b>Question 3:</b></font> Nous allons maintenant nous intéresser au comportement de ces modèles sur la base SONAR de la collection UCI (http://archive.ics.uci.edu/ml/index.php). Cette base contient 208 exemples en dimension 60 séparés par `,` et la dernière élément correspond à la classe de l'exemple.

    1. Télécharger la collection avec la fonction read_table de la librairie pandas (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). Les options nécessaires sont `sep=','` et `header=None`  
    2. Créer une liste de listes correspondant à la collection; pour cela initialiser la première liste et en parcourant chaque ligne de la matrice de données; créer une liste associée en remplaçant le dernier élément par `0` ou `+1` et insérer la dans la première liste. 
    Indication: Utiliser la fonction `loc`. 
    3. Écrire une fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable (indication: on pourra utiliser les fonctions `shuffle` de la librairie `random` et `train_test_split` de la librairie `sklearn.model_selection`)
    

Codage la Structure dataFrame

In [ ]:
class dataFrame:
    def __init__(self, df, target_column, class_type):
        if not isinstance(df, pd.DataFrame):
            raise ValueError("L'objet n'est pas un DataFrame")
        self.df = df
        self.target_column = target_column
        self.class_type = class_type
        self.num_data = self.df.shape[0]
        self.num_features = self.df.shape[1]
    
    def produce_collection(self):
        x = np.zeros((self.num_data, self.num_features - 1))
        y = np.zeros(self.num_data)
        
        features_idx = 0
        for i in range(self.num_features):
            if i == self.target_column:
                continue
            x[:, features_idx] = self.df[i]
            features_idx += 1
            
        if self.class_type == None:
            for i in range(self.num_data):
                y[i] = self.df[self.target_column][i]
        else:
            for i in range(self.num_data):
                if self.df[self.target_column][i] == self.class_type[0]: y[i] = 0
                elif self.df[self.target_column][i] == self.class_type[1]: y[i] = 1
                else: raise ValueError("Class type a plus de 2 classes")
        
        return collection(x, y)

Charger data frame

In [ ]:
df = pd.read_table('Data/Sonar/sonar.all-data', sep = ',', header = None)
df

Fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable

In [ ]:
# Fonction pour generer train et test random
def generate_train_test_random(self, test_size=0.25):
    data = list(zip(self.x, self.y))
    rd.shuffle(data)
    new_X = np.array([i for i, j in data])
    new_Y = np.array([j for i, j in data])
    x_train, x_test, y_train, y_test = train_test_split(new_X, new_Y, test_size=test_size)
    return collection(x_train, y_train), collection(x_test, y_test)

# Ajout de la fonction à la classe collection
collection.generate_train_test_random = generate_train_test_random

<font color='red'><b>Question 4:</b></font> Appliquer ces modèles sur cette base (on pourra prendre $MaxEp\approx1000$ et le pas d'apprentissage $\eta\approx0.1$) et en choisissant les bases Train et Test de façon aléatoire. Reporter l'erreur moyenne de ces modèles obtenues sur les exemples de donnés de "test"? 


In [ ]:
sonar_data_frame = dataFrame(df, 60, ['R', 'M'])
sonar_collection = sonar_data_frame.produce_collection()

sonar_train, sonar_test = sonar_collection.generate_train_test_random()


calculate_multiple_error_rates(adaline, sonar_train, sonar_test, 1, eta_001, 1000), calculate_multiple_error_rates(regression_logistique, sonar_train, sonar_test, 1, eta_001, 1000)

- Refaire l'opération 3 fois avec trois randomisations différentes. 

In [ ]:
calculate_multiple_error_rates(adaline, sonar_train, sonar_test, 3, eta_01, 1000, True), calculate_multiple_error_rates(regression_logistique, sonar_train, sonar_test, 3, eta_01, 1000, True)

  | Collection | Adaline     | Régression Logistique |
  |------------|-------------|-----------------------|
  |   SONAR (réplica 1)   |            |           |
  |   SONAR (réplica 2)   |           |         |
  |   SONAR (réplica 3)   |           |           |


## Partie 3: normalisation

Nous allons étudier l'impact de la nomralisation sur les prédictions. Pour cela nous considérons deux stratégies de normalisation communément utilisées dans la littérature:
* Stratégie <i>max</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la valeur maximale de cette caractéristiques
* Stratégie <i>norme</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la norme de ce vecteur.

Nous considérons ces trois autres collections de la base UCI:

        * https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
        * https://archive.ics.uci.edu/ml/datasets/spambase
        * https://archive.ics.uci.edu/ml/datasets/ionosphere

        

In [ ]:
# Importer dataset #
breast_cancer_df = pd.read_table('Data/BreastCancer/wdbc.data', header=None, sep=',')
ionosphere_df = pd.read_csv('Data/Ionosphere/ionosphere.data', header=None, sep=',')
spambase_df = pd.read_csv('Data/Spambase/spambase.data', header=None, sep=',')

# breast_cancer_df
# ionosphere_df
# spambase_df


<font color='red'><b>Question 5:</b></font> Ecrire une fonction qui prend en entrée la collection des données et qui retourne la collections normalisée suivant les stratégies <i>max</i> et <i>norme</i>. 

- Choisir tous les colonnes contenant des données numériques et les convertir en type `float` pour é viter les problème de division.
-  <i>Max</i>: Diviser par la valeur maximale en valeur absolu
-  <i>Norme</i>: Norme L2 

In [ ]:
def normalizer_collection(collection, strategies='max'):
    normalized_collection = collection.copy()
    
    numeric_cols = normalized_collection.select_dtypes(include=[np.number]).columns
    
    numeric_df = normalized_collection[numeric_cols].astype(float).copy()
    
    if strategies == 'max':
        max_value = np.max(np.abs(numeric_df), axis=0)
        max_value[max_value == 0] = 1 
        numeric_df = numeric_df / max_value
    
    elif strategies == 'norme': 
        l2_norm = np.linalg.norm(numeric_df, ord=2, axis=0)
        l2_norm[l2_norm == 0] = 1  
        numeric_df = numeric_df / l2_norm
        
    else:
        raise ValueError("Invalid normalization strategy")
    
    normalized_collection[numeric_cols] = numeric_df
    return normalized_collection


<font color='red'><b>Question 6:</b></font> Compléter les tableaux comparatifs suivants en repertant les erreurs moyennes sur 20 lancements des modèles de l'Adaline et de la Régression Logistique et pour les trois cas:




In [ ]:
num_iterations = 20
breast_cancer_df
# calculate_mean_error(breast_cancer_df, 1, ['M', 'B'], num_iterations, MaxEp, eta_001)
# calculate_mean_error(ionosphere_df, ionosphere_df.shape[1] - 1, ['g', 'b'], num_iterations, MaxEp, eta_001)
# calculate_mean_error(sonar_data_fram, sonar_data_fram.shape[1] - 1, class_type_sonar, num_iterations, MaxEp, eta_001)
# calculate_mean_error(spambase_df, spambase_df.shape[1] - 1, None, num_iterations, MaxEp, eta_001)


 '*' Les vecteurs ne sont pas normalisés
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |


In [ ]:
# Normalisation avec strategie norme #
breast_cancer_df_norme = normalizer_collection(breast_cancer_df, 'norme')
ionosphere_df_norme = normalizer_collection(ionosphere_df, 'norme')
sonar_data_fram_norme = normalizer_collection(sonar_data_fram, 'norme')
spambase_df_norme = normalizer_collection(spambase_df, 'norme')

print("Breast Cancer:")
calculate_mean_error(breast_cancer_df_norme, 1, ['M', 'B'], num_iterations, MaxEp, eta_001)
print("Ionosphere:")
calculate_mean_error(ionosphere_df_norme, ionosphere_df_norme.shape[1] - 1, ['g', 'b'], num_iterations, 1000, eta_001)
print("Sonar:")
calculate_mean_error(sonar_data_fram_norme, sonar_data_fram_norme.shape[1] - 1, class_type_sonar, num_iterations, 1000, eta_001)
print("Spambase:")
calculate_mean_error(spambase_df_norme, spambase_df_norme.shape[1] - 1, None, num_iterations, 1000, eta_001)



 
 $^n$ Normalisation suivant la stratégie <i>norme</i>
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |

  


In [ ]:
# Normalisation avec strategie max #
breast_cancer_df_max = normalizer_collection(breast_cancer_df)
spambase_df_max = normalizer_collection(spambase_df)
ionosphere_df_max = normalizer_collection(ionosphere_df)

print("Breast Cancer:")
calculate_mean_error(breast_cancer_df_max, 1, ['M', 'B'], num_iterations, MaxEp, eta_001)
print("Ionosphere:")
calculate_mean_error(ionosphere_df_max, ionosphere_df_max.shape[1] - 1, ['g', 'b'], num_iterations, MaxEp, eta_001)
print("Sonar:")
calculate_mean_error(sonar_data_fram, sonar_data_fram.shape[1] - 1, class_type_sonar, num_iterations, MaxEp, eta_001)
print("Spambase:")
calculate_mean_error(spambase_df_max, spambase_df_max.shape[1] - 1, None, num_iterations, MaxEp, eta_001)


 $^m$ Normalisation suivant la stratégie <i>max</i>
    
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |
